In [ ]:
import os

os.chdir(os.path.dirname(os.getcwd()))

In [ ]:
from pathlib import Path
import libs.utils
import libs.vars
import xarray

In [ ]:
ensemble = libs.vars.ensemble()

base_path = '_data/cmip6'
ukesm_file = f'/UKESM1-0-LL/siconc/siconc_SImon_UKESM1-0-LL_ssp585_r2i1p1f2_gn_201501-204912.nc'
ukesm_path = f'{base_path}{ukesm_file}'
not Path(ukesm_path).exists() and libs.utils.download_variable(**{
    'experiment_id': 'ssp585',
    'frequency': 'mon',
    'variable_id': 'siconc',
    'save_to_local': True,
    'source_id': 'UKESM1-0-LL',
    'table_id': 'SImon',
    'variant_label': 'r2i1p1f2'
})
ukesm_grid = xarray.open_mfdataset(
    paths=ukesm_path, 
    combine='by_coords',
    use_cftime=True
)

regrid_s2d = {
    'grid': ukesm_grid,
    'method': 'nearest_s2d',
    'copy_dims': ['i', 'j', 'longitude', 'latitude', 'vertices']
}
regrid_bil_s2d = {
    'grid': ukesm_grid,
    'method': 'bilinear',
    'extrap_method': 'nearest_s2d',
    'copy_dims': ['i', 'j', 'longitude', 'latitude', 'vertices']
}

experiments = [
    { 'experiment_id': 'historical', 'time_slice': slice('1980-01-01', '2015-01-01') },
    { 'experiment_id': 'ssp585', 'time_slice': slice('2015-01-01', '2101-01-01') }
]
variables = [
    { 'variable_id': 'siconc', 'frequency': 'mon', 'regrid_kwargs': regrid_s2d, 'table_id': 'SImon' },
    { 'variable_id': 'simass', 'frequency': 'mon', 'regrid_kwargs': regrid_s2d, 'table_id': 'SImon' },
    { 'variable_id': 'sithick', 'frequency': 'mon', 'regrid_kwargs': regrid_s2d, 'table_id': 'SImon' },
    { 'variable_id': 'sisnthick', 'frequency': 'mon', 'regrid_kwargs': regrid_s2d, 'table_id': 'SImon' },
    { 'variable_id': 'simpmass', 'frequency': 'mon', 'regrid_kwargs': regrid_s2d, 'table_id': 'SImon' },
    { 'variable_id': 'simpconc', 'frequency': 'mon', 'regrid_kwargs': regrid_s2d, 'table_id': 'SImon' },
    { 'variable_id': 'simprefrozen', 'frequency': 'mon', 'regrid_kwargs': regrid_s2d, 'table_id': 'SImon' },
    #{ 'variable_id': 'sipr', 'frequency': 'mon', 'regrid_kwargs': regrid_s2d, 'table_id': 'SImon' },
    #{ 'variable_id': 'sisnmass', 'frequency': 'mon', 'regrid_kwargs': regrid_s2d, 'table_id': 'SImon' },
    { 'variable_id': 'tos', 'frequency': 'mon', 'regrid_kwargs': regrid_s2d, 'table_id': 'Omon' },
    #{ 'variable_id': 'rls', 'frequency': 'mon', 'regrid_kwargs': regrid_bil_s2d, 'table_id': 'Emon' },
    #{ 'variable_id': 'rss', 'frequency': 'mon', 'regrid_kwargs': regrid_bil_s2d, 'table_id': 'Emon' },
    { 'variable_id': 'pr', 'frequency': 'mon', 'regrid_kwargs': regrid_bil_s2d, 'table_id': 'Amon' },
    { 'variable_id': 'prsn', 'frequency': 'mon', 'regrid_kwargs': regrid_bil_s2d, 'table_id': 'Amon' },
    { 'variable_id': 'tas', 'frequency': 'mon', 'regrid_kwargs': regrid_bil_s2d, 'table_id': 'Amon' },
    { 'variable_id': 'evspsbl', 'frequency': 'mon', 'regrid_kwargs': regrid_bil_s2d, 'table_id': 'Amon' }
]

In [ ]:
plot_variable = 'tos'

for e in experiments:
    for v in variables:
        for i, item in enumerate(ensemble):
            kwargs = { **e, **v }

            if item['source_id'] == 'EC-Earth3' and v['table_id'] in ['Amon', 'Emon']:
                kwargs['grid_label'] = 'gr'

            kwargs['source_id'] = item['source_id']
            kwargs['variant_label'] = item['variant_label']

            kwargs['force_write'] = False
            kwargs['process_files'] = True
            kwargs['save_to_local'] = True

            file = libs.utils.download_variable(**kwargs)
            if plot_variable == v['variable_id']:
                ensemble[i]['file'] = file

            print('-' * 20)

In [ ]:
import libs.plot
import warnings
warnings.filterwarnings('ignore')

ensemble_data = []
for item in ensemble:
    if (item['file'] == None):
        continue

    item_data = xarray.open_mfdataset(
        paths=item['file'], 
        combine='by_coords',
        use_cftime=True
    )
    ensemble_data.append({
        'data': item_data[plot_variable][6, :, :] - 273.15,
        'label': item['source_id']
    })

libs.plot.nstereo(
    ensemble_data,
    title=f'Ensemble {plot_variable}',
    colorbar_label=plot_variable,
    colormesh_kwargs={
        'cmap': 'RdBu_r',
        'extend': 'both',
        'levels': 21,
        'vmin': -10,
        'vmax': 10,
        'x': 'longitude', 
        'y': 'latitude'
    },
    shape=(1, 5)
)